<center>
    
### Data Mining Project 

### Project: US Presidential Impact on Afghanistan
### Team Members: Aditya Taori and Yuthika Shekhar
</center>

### Importing Libraries

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import re
import time
import string
from nltk.corpus import stopwords
import pandas as pd  
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis.gensim_models
import os
from nltk.tokenize import RegexpTokenizer,sent_tokenize,word_tokenize
import pandas as pd
import numpy as np
import pprint
from gensim.models import CoherenceModel

In [ ]:
### Setting Up Input Directories

In [ ]:
input_dir = "E:/UOR Notes/Data Mining/Assignments/US Presidential Impact/Output_Files/Tweets Data/"
input_file = "Osama bin Laden Killed.csv"
input_path = input_dir+input_file
event_name = "Osama Bin Laden Killed"
output_dir = "E:/UOR Notes/Data Mining/Assignments/US Presidential Impact/Output_Files/Topic Modelling/"

In [ ]:
### Reading Tweets Dataset

In [ ]:
tweets_df = pd.read_csv(input_path)
tweets_df.head()

In [ ]:
print(tweets_df.shape)
tweets_df.describe(include='all')

### Filtering out duplicate tweet ids

In [ ]:
unique_event_data = tweets_df.drop_duplicates(subset=["id_str"])
print(unique_event_data.shape)
unique_event_data.describe(include='all')

### Removing Retweets from Tweets Dataset 

In [ ]:
y = unique_event_data["text"].str.contains('^RT',regex=True)
y.value_counts()
retweets = unique_event_data[y]
indexes = retweets.index
indexes.tolist()
print(unique_event_data.shape)
unique_event_data = unique_event_data.drop(indexes)
print(unique_event_data.shape)


### Removing Hyperlinks from Tweets

In [ ]:
tweets_wo_hyperlinks = unique_event_data["text"].str.lower().replace('http\S+',"",regex=True)
#create_wordcloud(tweets_wo_hyperlinks)

### Removing Hashtags from Tweets

In [ ]:
tweets_wo_hashtags = tweets_wo_hyperlinks.replace("#[A-Za-z0-9]*","",regex=True)
#create_wordcloud(tweets_wo_hashtags)

### Removing Tags from Tweets

In [ ]:
tweets_wo_tags = tweets_wo_hashtags.replace("\@[A-Za-z0-9]*","",regex=True)
#create_wordcloud(tweets_wo_tags)

### Removing Stopwords from Tweets

In [ ]:
cachedStopWords = stopwords.words("english")
regexes= r'\b(' + r'|'.join(cachedStopWords) + r')\b\s*'
tweets_wo_sw= tweets_wo_tags.replace(regexes,"",regex=True)
#create_wordcloud(tweets_wo_sw)

In [ ]:
unique_event_data["Cleaned_Data"] = tweets_wo_sw

### Removing Empty Rows 

In [ ]:
unique_event_data['Cleaned_Data'].dropna(inplace=True)
print(unique_event_data.shape)
docs = np.array(unique_event_data['Cleaned_Data'])
docs = [doc for doc in docs if str(doc) != 'nan']

### Preprocessing the dataset 

In [ ]:
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    print(len(docs))
    for idx in range(len(docs)):
        #print(idx)
        #print(docs[idx])
                #docs[idx] = [[word.lower() for word in docs[idx].split()] for line in data] # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
        #print(docs[idx])
            # Remove numbers, but not words that contain numbers.
    docs_token = [[token.lower() for token in doc if not token.isdigit()] for doc in docs]

            # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]

            # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    return docs

In [ ]:
docs = docs_preprocessor(docs)
#docs

### Creating Bigrams

In [ ]:
bigram = Phrases(docs, min_count=10)
bigram

### Creating Trigrams

In [ ]:
trigram = Phrases(bigram[docs])

### Adding Bigrams and Trigrams to dictionaries

In [ ]:
#print(docs)

for idx in range(len(docs)):
    #print(idx)
    for token in bigram[docs[idx]]:
        if '_' in token:
                    # Token is a bigram, add to document.
            docs[idx].append(token)
            for token in trigram[docs[idx]]:
                if '_' in token:
                    # Token is a bigram, add to document.
                    docs[idx].append(token)


### Summary of Documents

In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

### Hyperparameter Tuning Code

In [ ]:
import time
# Set training parameters.
num_topics = range(5,6)
chunksize = 100 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

topic_hp_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])


for num in num_topics:
    print(num)
    inter_topic_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])
    # Make a index to word dictionary.
    if len(dictionary)>0:
        temp = dictionary[0]  # This is only to "load" the dictionary.
        start = time.time()
        id2word = dictionary.id2token
        model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize,alpha='auto', eta='auto', iterations=iterations, num_topics=num,passes=passes, eval_every=eval_every)
        end = time.time()
        print(end-start)
        time_taken = end-start

        Y = model.print_topics(num_words = 10)
        topic_words_list = []
        for i in range(0,len(Y)):
            #topic_words_list.append(html.Br())
            topic_words_list.append("Topic "+str(i+1)+": ")
            X = Y[i][1]
            a = X.split("+")
            words_str = ""
            for j in range(0,len(a)):
                p = a[j].split("*")[1].replace("\"","")
                words_str += p
            topic_words_list.append(words_str)

        perplexity = model.log_perplexity(corpus)
        coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        
        inter_topic_df["Event Name"] = [event_name]
        inter_topic_df["Number of Topics"] = [num]
        inter_topic_df["Coherence Score"] = [coherence_lda]
        inter_topic_df["Perplexity Score"] = [perplexity]
        inter_topic_df["Iterations"] = [iterations]
        inter_topic_df["Chunksize"] = [chunksize]
        inter_topic_df["Time Taken"] = [time_taken]
        inter_topic_df["Keywords in Topics"] = [topic_words_list]
        topic_hp_df = pd.concat([topic_hp_df,inter_topic_df])
        
        LDAvis_data_filepath = os.path.join(output_dir + event_name +'_sample_ldavis')
        # # this is a bit time consuming - make the if statement True
        # # if you want to execute visualization prep yourself
        if 1 == 1:
            LDAvis_prepared = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
            with open(LDAvis_data_filepath, 'wb') as f:
                pickle.dump(LDAvis_prepared, f)

        # load the pre-prepared pyLDAvis data from disk
        with open(LDAvis_data_filepath,'rb') as f:
            LDAvis_prepared = pickle.load(f)
        pyLDAvis.save_html(LDAvis_prepared, output_dir + event_name +'_sample_ldavis.html')


#topic_hp_df.to_csv(input_dir+"Topic_Modelling_Hyperparameter_Tuning.csv")
#topic_hp_df

In [ ]:
pyLDAvis.save_html(LDAvis_prepared, output_dir + event_name +'_sample_ldavis.html')

### Topic to Sentence Distribution Mapping

In [ ]:
tweets_df["Topic"] = "99"

for i in range(0,len(corpus)):
    #sent_topic_mapping["Sentence"].iloc[i] = p_df['DESCRIPTION'].iloc[i]
    print(i)
    topic_dist = model[corpus[i]]
    inter_topic = []
    topic_scores = []
    for j in range(0,len(topic_dist)):
        inter_topic.append(topic_dist[j][0])
        topic_scores.append(topic_dist[j][1])
    print(inter_topic)
    print(topic_scores)
    tweets_df["Topic"].iloc[i]= inter_topic[topic_scores.index(max(topic_scores))]+1

### Saving the data to CSV file

In [ ]:
filename = output_dir + event_name + "All_Data_Topic_Sentence_Distribution.csv"
tp_word_output = output_dir + event_name + "All_Data_Topic_Word_Distribution.csv"
tweets_df.to_csv(filename)
topic_hp_df.to_csv(tp_word_output)